In [14]:
# Import core TensorFlow modules
import tensorflow as tf
import numpy as np

In [15]:
# Modules required for file download and extraction
import os
import sys
import tarfile
from six.moves.urllib.request import urlretrieve
from scipy import ndimage

In [16]:
def maybe_download(filename, url, force=False):
  """Download a file if not present."""
  if force or not os.path.exists('/root/pipeline/datasets/notmnist/' + filename):
    filename, _ = urlretrieve(url + filename, '/root/pipeline/datasets/notmnist/' + filename)
    print('\nDownload complete for {}'.format(filename))
  else:
    print('File {} already present.'.format(filename))
  return '/root/pipeline/datasets/notmnist/' + filename

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('{} already present - don\'t need to extract {}.'.format(root, filename))
  else:
    print('Extracting data for {}. This may take a while. Please wait.'.format(root))
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(root[0:root.rfind('/') + 1])
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  print(data_folders)
  return data_folders

In [17]:
# Locations to download data:
url = 'http://yaroslavvb.com/upload/notMNIST/'

In [18]:
# Download two datasets
train_zip_path = maybe_download('notMNIST_small.tar.gz', url)

File notMNIST_small.tar.gz already present.


In [19]:
# Extract datasets
train_folders = maybe_extract(train_zip_path)

/root/pipeline/datasets/notmnist/notMNIST_small already present - don't need to extract /root/pipeline/datasets/notmnist/notMNIST_small.tar.gz.
['/root/pipeline/datasets/notmnist/notMNIST_small/A', '/root/pipeline/datasets/notmnist/notMNIST_small/B', '/root/pipeline/datasets/notmnist/notMNIST_small/C', '/root/pipeline/datasets/notmnist/notMNIST_small/D', '/root/pipeline/datasets/notmnist/notMNIST_small/E', '/root/pipeline/datasets/notmnist/notMNIST_small/F', '/root/pipeline/datasets/notmnist/notMNIST_small/G', '/root/pipeline/datasets/notmnist/notMNIST_small/H', '/root/pipeline/datasets/notmnist/notMNIST_small/I', '/root/pipeline/datasets/notmnist/notMNIST_small/J']


In [20]:
len(train_folders)

10

In [21]:
image_height = 28  # Pixel height of images
image_width = 28  # Pixel width of images
pixel_depth = 255.0  # Number of levels per pixel
expected_img_shape = (image_height, image_width)  # Black and white image, no 3rd dimension
num_labels = len(train_folders)

def load_image_folder(folder):
  """Load the data for a single image label."""
  
  # Create a list of image paths inside the folder  
  image_files = os.listdir(folder)
  # Create empty numpy array to hold data
  dataset = np.ndarray(shape=(len(image_files), image_height, image_width),
                         dtype=np.float32)
  num_images = 0  # Counter for number of successful images loaded
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      # Read in image pixel data as floating point values
      image_data = ndimage.imread(image_file).astype(float)
      # Scale values: [0.0, 255.0] => [-1.0, 1.0] 
      image_data = (image_data - pixel_depth / 2) / (pixel_depth / 2)
      if image_data.shape != expected_img_shape:
        print('File {} has unexpected dimensions: '.format(str(image_data.shape)))
        continue
      # Add image to the numpy array dataset
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- skipping this file and moving on.')
  
  # Trim dataset to remove unused space
  dataset = dataset[0:num_images, :, :]
  return dataset

In [28]:
def make_data_label_arrays(num_rows, image_height, image_width):
  """
  Creates and returns empty numpy arrays for input data and labels
  """
  if num_rows:
    dataset = np.ndarray((num_rows, image_height, image_width), dtype=np.float32)
    labels = np.ndarray(num_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def collect_datasets(data_folders):
  datasets = []
  total_images = 0
  for label, data_folder in enumerate(data_folders):
    # Bring all test folder images in as numpy arrays
    dataset = load_image_folder(data_folder)
    num_images = len(dataset)
    total_images += num_images
    datasets.append((dataset, label, num_images))
  return datasets, total_images

def merge_train_test_datasets(datasets, total_images, percent_test):
    num_train = total_images * (1.0 - percent_test)
    num_test = total_images * percent_test
    train_dataset, train_labels = make_data_label_arrays(num_train, image_height, image_width)
    test_dataset, test_labels = make_data_label_arrays(num_test, image_height, image_width)
    
    train_counter = 0
    test_counter = 0
    dataset_counter = 1
    for dataset, label, num_images in datasets:
      np.random.shuffle(dataset)
      if dataset_counter != len(datasets):
        n_v = num_images // (1.0 / percent_test)
        n_t = num_images - n_v
      else:
        # Last label, make sure dataset sizes match up to what we created
        n_v = len(test_dataset) - test_counter
        n_t = len(train_dataset) - train_counter
      train_dataset[train_counter: train_counter + n_t] = dataset[:n_t]
      train_labels[train_counter: train_counter + n_t] = label
      test_dataset[test_counter: test_counter + n_v] = dataset[n_t: n_t + n_v]
      test_labels[test_counter: test_counter + n_v] = label
      train_counter += n_t
      test_counter += n_v
      dataset_counter += 1
    return train_dataset, train_labels, test_dataset, test_labels

In [29]:
train_test_datasets, train_test_total_images = collect_datasets(train_folders)

('Could not read:', '/root/pipeline/datasets/notmnist/notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png', ':', IOError('cannot identify image file',), '- skipping this file and moving on.')
('Could not read:', '/root/pipeline/datasets/notmnist/notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png', ':', IOError('cannot identify image file',), '- skipping this file and moving on.')


In [30]:
train_dataset, train_labels, test_dataset, test_labels = \
  merge_train_test_datasets(train_test_datasets, train_test_total_images, 0.1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:41: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-int

In [31]:
len(train_dataset)

16851

In [32]:
# Convert data examples into 3-D tensors
num_channels = 1  # grayscale
def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_height, image_width, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels



In [33]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

In [34]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (16851, 28, 28, 1), (16851, 10))
('Test set', (1872, 28, 28, 1), (1872, 10))


In [39]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [42]:
batch_size = 100
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_height, image_width, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_height // 4 * image_width // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_rate):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    dropped = tf.nn.dropout(hidden, keep_rate)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [43]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.238878
Minibatch accuracy: 10.0%
Minibatch loss at step 50: 2.926146
Minibatch accuracy: 57.0%
Minibatch loss at step 100: 0.005914
Minibatch accuracy: 100.0%
Minibatch loss at step 150: 0.007855
Minibatch accuracy: 100.0%
Minibatch loss at step 200: 0.004339
Minibatch accuracy: 100.0%
Minibatch loss at step 250: 0.005817
Minibatch accuracy: 100.0%
Minibatch loss at step 300: 0.008423
Minibatch accuracy: 100.0%
Minibatch loss at step 350: 0.013187
Minibatch accuracy: 100.0%
Minibatch loss at step 400: 0.007962
Minibatch accuracy: 100.0%
Minibatch loss at step 450: 0.007794
Minibatch accuracy: 100.0%
Minibatch loss at step 500: 0.008829
Minibatch accuracy: 100.0%
Minibatch loss at step 550: 0.021692
Minibatch accuracy: 100.0%
Minibatch loss at step 600: 0.006766
Minibatch accuracy: 100.0%
Minibatch loss at step 650: 0.018554
Minibatch accuracy: 100.0%
Minibatch loss at step 700: 0.015392
Minibatch accuracy: 100.0%
Minibatch loss at step 750: 0.008